In [7]:
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import os
import chromedriver_autoinstaller

In [8]:
# URL of the ShareChat page
url = "https://sharechat.com/trending/Hindi"
chromedriver_autoinstaller.install()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-popup-blocking')
driver = webdriver.Chrome(options=chrome_options)

driver.get(url)
if not os.path.exists('./imgs'):
    os.makedirs('./imgs')

scroller = driver.find_element(By.XPATH,
                               "//div[@class='infinite-list-wrapper']")

posts = scroller.find_elements(
    By.XPATH, './/div[@data-cy="image-post"] | //div[@data-cy="video-post"] | //div[@data-cy="gif-post"]')
print(len(posts))
# time.sleep(10)

for post in posts:
    try:
        # get the data-post-ph attribute of post
        post_ph = post.get_attribute('data-post-ph')
        print("Post PH:", post_ph)
        # Locate the elements containing the information
        author_element = post.find_element(
            By.CSS_SELECTOR, 'strong[data-cy="author-name"]')
        author_name = author_element.text

        # disabling content collection

        # imgElement = post.find_element(
        #     By.XPATH, './/div[contains(@class, "Pos(a)") and contains(@class, "Start(0)") and contains(@class, "W(100%)") and contains(@class, "T(0)") and contains(@class, "H(100%)")]')

        # screenshot
        # check if image already exists and add a number to it
        # num = 0
        # while os.path.exists(f'./imgs/{author_element.text}_{num}.png'):
        #     num += 1

        # scroll into view
        # driver.execute_script("arguments[0].scrollIntoView();", author_element)

        # time.sleep(3)
        # print("Taking screenshot")
        # imgElement.screenshot(
        #     f'./imgs/{author_element.text}_{num}.png')

        # div[@class="H(100%) Pstart($xs) Fxg(1) Miw(0)"]
        topDetailsDiv = post.find_element(
            By.XPATH, './/div[@class="H(100%) Pstart($xs) Fxg(1) Miw(0)"]')

        # find direct div inside it
        topDetailsDiv = topDetailsDiv.find_element(
            By.XPATH, './div[@class="H(100%) Ta(start) D(f) Jc(c) Ai(fs) Fxd(c)"]')

        # direct divs inside it
        innerDivs = topDetailsDiv.find_elements(
            By.XPATH, './div')
        text = innerDivs[1].text.split('•')
        years_before = text[1].strip()
        number_of_views = text[0].strip()

        post_caption = post.find_element(
            By.XPATH, './/div[@data-cy="post-caption"]')

        print("Author Name:", author_name)
        print("Number of Views:", number_of_views)
        print("Years Before:", years_before)
        print("Post Caption:", post_caption.text)

        # open new page comments
        commentLink = f"https://sharechat.com/comment/{post_ph}"
        print("Number of windows:", len(driver.window_handles))

        original_window = driver.current_window_handle

        driver.execute_script(f"window.open('{commentLink}');")
        time.sleep(1)

        new_window = [
            window for window in driver.window_handles if window != original_window][0]
        driver.switch_to.window(new_window)
        time.sleep(1)

        topBar = driver.find_element(
            By.XPATH, '//ul[@class="List(n)  D(f) Ai(c) W(100%) H(100%) "]')
        # click on 3rd li
        toFindinLike = "लाइक"
        idx = 0

        topEls = topBar.find_elements(By.XPATH, './li')
        for idx, li in enumerate(topEls):
            if toFindinLike in li.text:
                break

        likeCount = int(topEls[idx].text.replace(toFindinLike, '').strip())
        print("Likes: ", likeCount)
        # click
        topEls[idx].click()

        mainDiv = driver.find_element(
            By.XPATH, '//div[@class="Ovy(a) Fxg(1) W(100%) Maw(600px) M(a)"]')

        # loadedlikes = number of a
        loaded_likes = len(mainDiv.find_elements(
            By.XPATH, './/a[@data-cy="avatar-tag"]'))
        pbar = tqdm(total=likeCount)
        scroll_increment = 200  # The amount by which to increment the scroll each time
        current_scroll_position = 0  # Keep track of the current scroll position

        checkFinishTimer = 10
        startTimer = time.time()
        while loaded_likes < likeCount:
            driver.execute_script(
                f"arguments[0].scrollTop = {current_scroll_position}", mainDiv)
            current_scroll_position += scroll_increment

            if time.time() - startTimer > checkFinishTimer:
                listEls = mainDiv.find_elements(
                    By.XPATH, './/a[@data-cy="avatar-tag"]')
                loaded_likes = len(listEls)

                pbar.update(loaded_likes - pbar.n)
                startTimer = time.time()

        pbar.close()
        time.sleep(3)

        # get the list of all the users
        listEls = mainDiv.find_elements(
            By.XPATH, './/a[@data-cy="avatar-tag"]')

        # in this, strong tag with data-cy="author-name" is the name of the user, get it
        # and add it to a list
        users = []
        for li in listEls:
            users.append(li.find_element(
                By.XPATH, './/strong[@data-cy="author-name"]').text)

        print("Users:", users)
        print("Number of users:", len(users))
        driver.close()
        driver.switch_to.window(original_window)

    except Exception as e:
        print("An error occurred:", str(e))
        continue

20
Post PH: 1lbvePR
Author Name: Manorama bansal
Number of Views: 1.4K ने देखा
Years Before: 4 घंटे पहले
Post Caption: #👉 Saturday Motivation
Number of windows: 1
Likes:  30


45it [00:10,  4.49it/s]               


Users: ['Vier Singh Arya', 'a b sandhu', 'neetu bala', 'ashu', 'k .r nager', 'sanny singh', 'Anjali', 'мαнαvεεя нιη∂υsтαηι', 'User', 'deep vishwash', 'ShareChatUser', 'ShareChatUser', 'ShareChatUser', 'ShareChatUser', 'ShareChatUser', 'ShareChatUser', 'ShareChatUser', 'Anju Chauhan', 'ShareChatUser', 'Jayashree Rathod', 'Mamta Shukla', 'ShareChatUser', 'ShareChatUser', 'Mukesh Kumawat', 'Mansingh Gangwal', 'Geet Neupane', 'shyam Singh', 'Naina Ram choudhary', 'Shobhan Garg', 'Sureet kumar kashyap', 'ShareChatUser', 'bhola marchent', 'ShareChatUser', 'ShareChatUser', 'User', 'kavita Ganote', 'ShareChatUser', 'ShareChatUser', 'ShareChatUser', 'Ankita Pandey', 'ShareChatUser', 'ShareChatUser', 'Vivek Malviy', 'ruhi yaduvanshi', 'ShareChatUser']
Number of users: 45
Post PH: 1Rb8Xwe7
Author Name: Kalyani Kumari
Number of Views: 141.4K ने देखा
Years Before: 5 घंटे पहले
Post Caption: #🙏राम राम जी #🌞 Good Morning🌞 #❤️शुभकामना सन्देश
Number of windows: 1
Likes:  1344


  0%|          | 0/1344 [00:00<?, ?it/s]

KeyboardInterrupt: 